# Customer Satisfaction

## 데이터분석과 시각화, 머신러닝 알고리즘으로 은행 고객 만족도 예측하기

기업의 C-suites(중역)에게 고객의 만족도는 매우 중요한 요소입니다. 그러나 불행하게도 고객은 c-suites 근처에 있지도 않고, 그들의 불만을 잘 토로하지도 않습니다.

그런 이유로 은행에서는 이미 수집한 데이터를 토대로 고객의 만족도를 예측하여 늦기 전에 그들의 만족도 위한 능동적인 행동을 취하고 싶어합니다.

여러분은 수백개의 익명처리된 변수를 이용하여 고객의 만족도를 예측해주세요.

## 파일 구성
 - **train.csv**: 학습 데이터
 - **test.csv**: 평가 데이터
 - **submission.csv**: 제출 데이터

## 특성 설명
 - 모든 feature는 익명처리되어 있습니다.
 - `TARGET`: 0->만족, 1->불만족, 레이블 (종속변수, 타겟변수)

## 평가 측도
 - **`auc`**: 높을수록 좋음

# 필수 모듈

In [1]:
#roc_curve 쓰면 auc나옴
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
!pip install jaen

You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [3]:
# ID@jaen.com
# ID를 문자열로 입력해주세요.
#var3 -> 은행지역
#var5 ->고객이 나이
#var38 -> 대출 금액
#imp -> 수입 
#saldo -> 통장 잔액

ID = ''
email = ID + '@jaen.com'

In [4]:
from JAEN.project import Project
pjt = Project('고객 만족도 예측', 
              '한림대-파이썬을활용한-머신러닝프로젝트',        
              '1차수 A반',       
              email)

# 데이터 다운로드

In [5]:
!wget -N --http-user=mysuni --http-passwd=mysuni1! http://sk.jaen.kr:8080/hl_customer_satis.zip

--2022-02-17 13:46:06--  http://sk.jaen.kr:8080/hl_customer_satis.zip
sk.jaen.kr (sk.jaen.kr) 해석 중... 49.247.134.238
다음으로 연결 중: sk.jaen.kr (sk.jaen.kr)|49.247.134.238|:8080... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 401 Unauthorized
선택한 인증: Basic realm="Authentication required."
sk.jaen.kr에 기존 연결 재활용:8080.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 200 OK
길이: 7181776 (6.8M) [application/zip]
저장 위치: `hl_customer_satis.zip'

hl_customer_satis.z 100%[===================>]   6.85M  1.03MB/s    /  5.9s    

2022-02-17 13:46:12 (1.17 MB/s) - `hl_customer_satis.zip' 저장함 [7181776/7181776]



In [6]:
import zipfile

# 데이터 폴더
DATA_DIR = './data'

with zipfile.ZipFile("hl_customer_satis.zip" , "r") as zip_ref:
    zip_ref.extractall(DATA_DIR)

# Baseline

In [285]:
# 데이터 로드
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
submission = pd.read_csv('./data/submission.csv')

In [286]:
train

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
3,8,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
4,10,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76015,151829,2,48,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60926.490000,0
76016,151830,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,118634.520000,0
76017,151835,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,74028.150000,0
76018,151836,2,25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,84278.160000,0


In [287]:
train.nunique()

ID                         76020
var3                         208
var15                        100
imp_ent_var16_ult1           596
imp_op_var39_comer_ult1     7551
                           ...  
saldo_medio_var44_hace3       33
saldo_medio_var44_ult1       141
saldo_medio_var44_ult3       141
var38                      57736
TARGET                         2
Length: 371, dtype: int64

In [288]:
test.head()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
0,2,2,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40532.10
1,5,2,35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45486.72
2,6,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46993.95
3,7,2,24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,187898.61
4,9,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73649.73


In [289]:
submission.head()

,ID,TARGET
0,2,0
1,5,0
2,6,0
3,7,0
4,9,0


In [290]:
train['TARGET'].value_counts()#0 -> 만족 , 1 -> 불만족

0    73012
1     3008
Name: TARGET, dtype: int64

In [291]:
train.shape

(76020, 371)

In [292]:
for col in train.columns:
  #print(train[col])
  if col == 'TARGET':
    continue
  if (test[col].nunique() == 1):
      print(col)
      train = train.drop(columns = col, axis=1)
    

ind_var2_0
ind_var2
ind_var27_0
ind_var28_0
ind_var28
ind_var27
ind_var41
ind_var46_0
ind_var46
num_var27_0
num_var28_0
num_var28
num_var27
num_var41
num_var46_0
num_var46
saldo_var28
saldo_var27
saldo_var41
saldo_var46
delta_imp_reemb_var33_1y3
delta_imp_trasp_var17_out_1y3
delta_num_reemb_var33_1y3
delta_num_trasp_var17_out_1y3
imp_amort_var18_hace3
imp_amort_var34_hace3
imp_reemb_var13_hace3
imp_reemb_var17_hace3
imp_reemb_var33_hace3
imp_reemb_var33_ult1
imp_trasp_var17_out_hace3
imp_trasp_var17_out_ult1
imp_trasp_var33_out_hace3
num_var2_0_ult1
num_var2_ult1
num_reemb_var13_hace3
num_reemb_var17_hace3
num_reemb_var33_hace3
num_reemb_var33_ult1
num_trasp_var17_out_hace3
num_trasp_var17_out_ult1
num_trasp_var33_out_hace3
saldo_var2_ult1
saldo_medio_var13_medio_hace3
saldo_medio_var29_hace3


In [293]:
train.head()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
3,8,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
4,10,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0


In [294]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76020 entries, 0 to 76019
Columns: 326 entries, ID to TARGET
dtypes: float64(108), int64(218)
memory usage: 189.1 MB


In [295]:
s = train.T.duplicated()
idx = s[s==True].index
train.drop(idx,axis=1)

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
3,8,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
4,10,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76015,151829,2,48,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60926.490000,0
76016,151830,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,118634.520000,0
76017,151835,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,74028.150000,0
76018,151836,2,25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,84278.160000,0


In [318]:
#train.drop('ID', axis = 1, inplace = True)
train

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
1,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
2,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
3,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
4,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76015,2,48,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60926.490000,0
76016,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,118634.520000,0
76017,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,74028.150000,0
76018,2,25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,84278.160000,0


In [348]:
train['var3'].replace(-999999, 2, inplace=True)

In [349]:
print(train['var3'].value_counts( )[:10])

2     74281
8       138
9       110
3       108
1       105
13       98
7        97
4        86
12       85
6        82
Name: var3, dtype: int64


In [296]:
for col in test.columns:
  #print(train[col])
  if (test[col].nunique() == 1):
      print(col)
      test = test.drop(columns = col, axis=1)

ind_var2_0
ind_var2
ind_var27_0
ind_var28_0
ind_var28
ind_var27
ind_var41
ind_var46_0
ind_var46
num_var27_0
num_var28_0
num_var28
num_var27
num_var41
num_var46_0
num_var46
saldo_var28
saldo_var27
saldo_var41
saldo_var46
delta_imp_reemb_var33_1y3
delta_imp_trasp_var17_out_1y3
delta_num_reemb_var33_1y3
delta_num_trasp_var17_out_1y3
imp_amort_var18_hace3
imp_amort_var34_hace3
imp_reemb_var13_hace3
imp_reemb_var17_hace3
imp_reemb_var33_hace3
imp_reemb_var33_ult1
imp_trasp_var17_out_hace3
imp_trasp_var17_out_ult1
imp_trasp_var33_out_hace3
num_var2_0_ult1
num_var2_ult1
num_reemb_var13_hace3
num_reemb_var17_hace3
num_reemb_var33_hace3
num_reemb_var33_ult1
num_trasp_var17_out_hace3
num_trasp_var17_out_ult1
num_trasp_var33_out_hace3
saldo_var2_ult1
saldo_medio_var13_medio_hace3
saldo_medio_var29_hace3


In [298]:
test.drop(idx,axis=1)

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
0,2,2,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40532.100000
1,5,2,35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45486.720000
2,6,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46993.950000
3,7,2,24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,187898.610000
4,9,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73649.730000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75813,151831,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40243.200000
75814,151832,2,26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,146961.300000
75815,151833,2,24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,167299.770000
75816,151834,2,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016


In [319]:
test.drop('ID', axis = 1, inplace = True)

In [351]:
train['var3'].replace(-999999, 2, inplace=True)

In [352]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

X = train.iloc[:, :-1]
Y = train['TARGET']


x_train, x_test, y_train ,y_test = train_test_split(X,Y, random_state=0)



In [354]:
params = { 'n_estimators' : [100],
           'max_depth' : [6, 8, 10, 12],
           'min_samples_leaf' : [8, 12, 18],
           'min_samples_split' : [8, 16, 20]
            }

model = RandomForestClassifier(random_state = 0, n_jobs = -1)

grid_cv = GridSearchCV(model, param_grid = params, cv = 3)
grid_cv.fit(x_train, y_train)

print('최적 하이퍼 파라미터: ', grid_cv.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))

KeyboardInterrupt: 

In [355]:

#위의 결과로 나온 최적 하이퍼 파라미터로 다시 모델을 학습하여 테스트 세트 데이터에서 예측 성능을 측정
model = RandomForestClassifier(n_estimators = 200, 
                                max_depth = 6,
                                min_samples_leaf = 8,
                                min_samples_split = 8,
                                random_state = 0,
                                n_jobs = -1)
model.fit(X, Y)



RandomForestClassifier(max_depth=6, min_samples_leaf=8, min_samples_split=8,
                       n_estimators=200, n_jobs=-1, random_state=0)

# XGBoost

In [232]:
# 객체 생성, 일단은 트리 100개만 만듦
xgb_model = XGBClassifier(n_estimators=100)

# 후보 파라미터 선정
params = {'max_depth':[5,7], 'min_child_weight':[1,3], 'colsample_bytree':[0.5,0.75]}

# gridsearchcv 객체 정보 입력(어떤 모델, 파라미터 후보, 교차검증 몇 번)
gridcv = GridSearchCV(xgb_model, param_grid=params, cv=3)

# 파라미터 튜닝 시작
gridcv.fit(x_train, y_train, early_stopping_rounds=30, eval_metric='auc', eval_set=[(x_test, y_test)])

#튜닝된 파라미터 출력
print(gridcv.best_params_)

최적 하이퍼 파라미터:  {'max_depth': 6, 'min_samples_leaf': 8, 'min_samples_split': 8, 'n_estimators': 100}
최고 예측 정확도: 0.9605


In [361]:

# 1차적으로 튜닝된 파라미터를 가지고 객체 생성
xgb_model = XGBClassifier(n_estimators=3000, objective='binary:logistic', learning_rate=0.02, max_depth=7, min_child_weight=1, colsample_bytree=0.5, reg_alpha=0.03)

# 학습
xgb_model.fit(x_train, y_train, early_stopping_rounds=200, eval_metric='auc', eval_set=[(x_test, y_test)])


/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[0]	validation_0-auc:0.74438
[1]	validation_0-auc:0.81359
[2]	validation_0-auc:0.82288
[3]	validation_0-auc:0.82828
[4]	validation_0-auc:0.83051
[5]	validation_0-auc:0.82606
[6]	validation_0-auc:0.82286
[7]	validation_0-auc:0.82808
[8]	validation_0-auc:0.82942
[9]	validation_0-auc:0.83076
[10]	validation_0-auc:0.82840
[11]	validation_0-auc:0.82754
[12]	validation_0-auc:0.82880
[13]	validation_0-auc:0.83031
[14]	validation_0-auc:0.83116
[15]	validation_0-auc:0.83010
[16]	validation_0-auc:0.83067
[17]	validation_0-auc:0.82876
[18]	validation_0-auc:0.82813
[19]	validation_0-auc:0.82881
[20]	validation_0-auc:0.82975
[21]	validation_0-auc:0.83008
[22]	validation_0-auc:0.83021
[23]	validation_0-auc:0.83002
[24]	validation_0-auc:0.82934
[25]	validation_0-auc:0.82888
[26]	validation_0-auc:0.83013
[27]	validation_0-auc:0.82886
[28]	validation_0-auc:0.83050
[29]	validation_0-auc:0.83088
[30]	validation_0-auc:0.83131
[31]	validation_0-auc:0.83162
[32]	validation_0-auc:0.83228
[33]	validation_0-au

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5,
              enable_categorical=False, gamma=0, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.02, max_delta_step=0, max_depth=7,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=3000, n_jobs=16, num_parallel_tree=1,
              predictor='auto', random_state=0, reg_alpha=0.03, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [362]:
from sklearn.metrics import roc_auc_score

pred_proba = model.predict_proba(x_test)[:,1]
print('random forest ROC AUC Score: ', roc_auc_score(y_test,pred_proba))

pred_proba2 = xgb_model.predict_proba(x_test)[:,1]
print('xgb ROC AUC Score: ', roc_auc_score(y_test,pred_proba2))

random forest ROC AUC Score:  0.8287489843633529
xgb ROC AUC Score:  0.8471235666310419


In [363]:
pred_p = xgb_model.predict_proba(test)[:,1]
#0.4*model.predict_proba(test)[:,1]+0.6*

In [364]:
submission['TARGET'] = pred_p

In [365]:
pjt.submit(submission)

파일을 저장하였습니다. 파일명: submission-17-29-20.csv
제출 여부 :success
오늘 제출 횟수 : 33
제출 결과:0.8371


# 이제부터 자유롭게 분석, 예측을 진행하시면 됩니다.